<a href="https://colab.research.google.com/github/blancavazquez/CursoDatosMasivosI/blob/master/notebooks/4c_elementos_distintos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Algoritmo de Flajolet-Martin
En esta libreta programaremos el algoritmo Flajolet-Martin  para estimar el número de elementos distintos en un flujo de datos.

La idea detrás de este algoritmo es que entre más elementos diferentes haya en el flujo de datos, más valores _hash_ diferentes veremos y es más probable que alguno de estos valores tenga una representación binaria que termine con un mayor número de ceros consecutivos.

En particular, sea $tam(x)$ el número de 0s al final de la cadena correspondiente al valor _hash_ del elemento $x$, se inicializa un arreglo de bits de tamaño $L$ con ceros y se pone a 1 el bit de la posición $tam(x)$ para cada $x$ del flujo. Sea $r$ la primera posición del arreglo de bits cuyo valor es cero, un estimador del número de elementos en el flujo de datos es $\frac{2^{r}}{\phi}$, donde $\phi \approx 0.77351$ es un factor de corrección.

In [1]:
#Extracción de número de 0s al final de la cadena
hash = 16
print("Valor hash en binario: (str)", bin(hash))
print("Extracción del valor binario: ", bin(hash)[2:])
print("Dando la 'vuelta' al valor binario: ",bin(hash)[2:][::-1])
print("¿Cuántos ceros tiene:?", bin(hash)[2:][::-1].find('1'))

Valor hash en binario: (str) 0b10000
Extracción del valor binario:  10000
Dando la 'vuelta' al valor binario:  00001
¿Cuántos ceros tiene:? 4


In [2]:
import numpy as np
np.random.seed(2021) # para reproducibilidad

class ConteoProbabilista:  
  def __init__(self, n_cubetas, primo, n_bits=64):
    self.primo = primo  
    self.n_cubetas = n_cubetas
    self.a = np.random.randint(1, self.primo - 1) #iteraramos con diferentes funciones hash
    self.b = np.random.randint(0, self.primo - 1)
    self.bitmap = np.zeros(n_bits, dtype=bool)

  def __call__(self, x):
    hash = ((self.a * x + self.b) % self.primo) % self.n_cubetas #función hash universal
    i = bin(hash)[2:][::-1].find('1') #nos regresa el número de 0s al final de la cadena
    self.bitmap[i] = 1 #mantenemos el número máx de 0s que ha visto

  def cardinalidad(self): #función para calcular el número de elementos distintos
    r = np.argwhere(self.bitmap == 0)[0]
    r = (2**r) / 0.77351
    return r

Definimos una clase que realiza varias estimaciones, las divide en grupos pequeños, obtiene la mediana de las estimaciones de cada grupo y toma el promedio de las medianas como estimación final.

In [3]:
class EstimadorElementosDistintos:
  def __init__(self, n_cubetas, n_grupos, n_funciones, primo, n_bits):
    self.n_grupos = n_grupos
    self.n_funciones = n_funciones
    self.estimadores = [] #es una lista de lista que guarda todos los estimadores (n_grupos ->n_funciones )
    for i in range(self.n_grupos): #se itera en cada uno de los grupos
      func = []
      for j in range(self.n_funciones): #se generan n funciones por cada grupo
        func.append(ConteoProbabilista(n_cubetas, primo, n_bits)) #se generan las diferentes estimaciones
      self.estimadores.append(func)
    self.conteos = np.zeros((self.n_grupos, self.n_funciones)) #matriz [n_grupos,nfunciones], va estar asociada a una función distinta hash

  def __call__(self, x):
    for i in range(self.n_grupos):
      for j in range(self.n_funciones):  
        self.estimadores[i][j](x)
        self.conteos[i, j] = self.estimadores[i][j].cardinalidad()
      
  def cardinalidad(self):
    return np.mean(np.median(self.conteos, axis=1)) ##Se calcula la mediana  de cada uno de los grupos pequeños y luego se calcula el promedio

Generamos números aleatorios.

In [4]:
import numpy as np

X = np.random.randint(0,100000, size=1000000)
print("Hay {0} elementos distintos".format(np.unique(X).size))

Hay 99995 elementos distintos


Instanciamos nuestra clase y estimamos elementos distintos.

In [5]:
est = EstimadorElementosDistintos(n_cubetas=1000000, n_grupos=5, n_funciones=5, primo=4294967291, n_bits=64)

for i,x in enumerate(X):
  est(x)

#Recordemos que: cada uno de los grupos nos darán estimaciones distintas
print(u'Real = {0} Estimación = {1} '.format(np.unique(X[:i+1]).size, est.cardinalidad()))

Real = 99995 Estimación = 88961.74580806971 
